
# User Guide: AI Recipe Chatbot Application

## Introduction
Welcome to the AI Recipe Chatbot Application! This innovative tool allows you to generate detailed recipes and accompanying images by simply providing a list of ingredients. The application uses state-of-the-art AI models for text and image generation, deployed on Hugging Face Spaces with a user-friendly interface built using Gradio.

---

## Getting Started

### 1. System Requirements
- **Operating System:** Windows/macOS/Linux
- **Python Version:** 3.8 or higher
- **Internet Connection:** Required for API requests

### 2. Installation and Setup
- Clone the application from the repository: `git clone https://github.com/joethi/SDS-CP017-ai-recipe-wizard.git`
- Navigate to the project directory: `cd notebooks/jothi-thondiraj`
- Install dependencies
- Set up your Hugging Face API token as an environment variable:
  ```bash
  export API_Token_HF_AIrecipe="your_huggingface_api_token"
  ```

### 3. Launching the Application
- Run the script to start the Gradio interface:
  ```bash
  python app.py
  ```
- Open your browser and go to the local server link displayed in the terminal (e.g., `http://127.0.0.1:7860`).

---

## Navigating the Interface

### 1. Chat Interface

- The main interaction point for users.
- Input ingredients separated by commas in the chat box to generate a recipe and its image.
- Click the ingredients displayed in the user interface to get started.
- Here is the image of how the user interface would look like:

    <!-- ![Chat Interface Example](./Outputs/UI.png) -->
    <img src="./Outputs/UI.png" alt="Chat Interface Example" width="900" height="500"/>


### 2. Recipe Image Section
- Displays the generated recipe and the image once processing is complete.

### 3. Examples Section
- Provides pre-loaded ingredient examples to explore the application’s capabilities.
---

## Using the Application

### 1. Enter Ingredients
- Type or copy-paste the list of ingredients into the chat box.
- Example: `chicken, rice, tomatoes, onions, spices`

### 2. Generate Recipe and Image
- The chatbot will generate a detailed recipe, including a title, steps, and an image of the dish.

### 3. View Results
- The generated recipe text will appear in the chat interface.
- The recipe image will be displayed in the Recipe Image section.

### 3. Example Results
- For the ingredients: `chicken, rice, tomatoes, onions, spices`, here is the example output:

    <img src="./Outputs/spicy_chicken.png" alt="Chat Interface Example" width="890" height="490"/>

- For dessert lovers, given the ingredients: `cinnamon, rice, milk`, here is the example output:

     <img src="./Outputs/Cinnamon_Rice_pudding.png" alt="Chat Interface Example" width="900" height="500"/>  

- Finally, for vegans and vegetarians, here is the recipe with the ingredients: `tofu, noodles, soy sauce`

    <img src="./Outputs/TOFU_RECIPE.png" alt="Chat Interface Example" width="900" height="500"/>


    
---

## Troubleshooting

### 1. Common Issues
- **No API Token:** Ensure the `API_Token_HF_AIrecipe` environment variable is set correctly.
- **Connection Error:** Check your internet connection.
- **Longer Wait Times** Since the models are currently limited to simple open source models, 
  the running time is a bit long sometimes. But it is worth the wait to see the delicious recipes!
  If you want to use the faster models such as OpenAI models, please look at the notebook: 
  `JothiThondiraj-AIRecipeWizard-openai.ipynb`

### 2. Error Messages
- **Error generating recipe:** Verify the Hugging Face model name and token.
- **Error generating image:** Ensure the image generation model is accessible.

---

## Tips for Best Results

1. Use detailed ingredient lists for more specific recipes.
2. Keep ingredient names simple to avoid confusion for the AI.
3. Experiment with the example inputs to explore variations.

---

## Want a ready-to-go app? we got your back use HuggingFace Spaces:

1. Visit the [Hugging face Spaces](https://huggingface.co/spaces). 
2. In the search bar, type `jothi3112/Chef_Jothi_AI_Magic` and enter to find my space.
3. Alternatively you can use the [Link](https://huggingface.co/spaces/jothi3112/Chef_Jothi_AI_Magic) to access the application.

---

## Feedback and Support

### 1. Feedback
- Use the feedback option in the application to report bugs or suggest features.

### 2. Contact Support
- Email: nasaaimer2014@gmail.com

---

Enjoy creating recipes with the power of AI!



In [1]:
# pip install openai

In [2]:
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv, dotenv_values
import requests
from PIL import Image
from io import BytesIO
import gradio as gr
import json
import re

/opt/anaconda3/envs/SDS_Projects/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load variables from .env file
load_dotenv()

True

In [4]:
def generate_recipe(ingredients: str, model_name_recipe, client):
    # Generate recipe
    prompt = f"Generate a recipe using the following ingredients: {ingredients} and make sure it has maximum of 500 tokens. Start with title as Recipe: Recipe Name. Add cooking time after ingredients"
    completion = client.chat.completions.create(
        model=model_name_recipe,
        messages=[
            {"role": "developer", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=500
    )
    recipe_text = completion.choices[0].message.content
    print("type(recipe_text):",type(recipe_text))
    print("recipe_text:",recipe_text)
    # response_json = json.loads(recipe_text)
    # print("response_json",response_json)
    return recipe_text

In [5]:
def generate_image(recipe_title: str, model_name_image: str, client) -> Image.Image:
    # Generate image based on recipe
    recipe_prompt = f"An image of the recipe: {recipe_title}, showing vibrant colors and delicious presentation. Do not use any text in the image"
    response = client.images.generate(
        model=model_name_image,
        prompt=recipe_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url

    # Download the image
    image_response = requests.get(image_url)
    if image_response.status_code == 200:
        image = Image.open(BytesIO(image_response.content))
    else:
        raise Exception(f"Failed to download the image. Status code: {image_response.status_code}")
    return image_url

In [6]:
def generate_recipe_and_image(message: str, history=None):
    # Initialize OpenAI client
    your_openapi_key = ""
    client = OpenAI(api_key=os.environ.get(your_openapi_key))
    model_name_recipe = "gpt-4o"  # Replace with the LLM model of your choice
    model_name_image = "dall-e-3"  # Replace with the image generation model of your choice  
    # Generate recipe
    recipe_text = generate_recipe(message, model_name_recipe, client)
    # Extract recipe title from generated recipe text
    # lines = recipe_text.split("\n")
    # for line in lines:
    #     if line.lower().startswith("recipe:"):  # Case-insensitive match
    #         recipe_title = line.replace("Recipe:", "").strip()  # Extract and clean title
    #         break 
    # print("recipe_title",recipe_title) 
    # Regex pattern to match the title after "Recipe:"
    pattern = r"(?i)\brecipe:\s*(.*)"
    # Search for the pattern in the response text
    match = re.search(pattern, recipe_text)

    if match:
        recipe_title = match.group(1).strip()
        print("Extracted Title:", recipe_title)
    else:
        print("Title not found.")
        recipe_title = recipe_text
    # print("recipe_text:",recipe_text["recipe"]) 
    # Generate image for the recipe
    recipe_image = generate_image(recipe_title, model_name_image, client)
    
    return recipe_text, 
    # return recipe_text

In [9]:
# Gradio Chat Interface
chat_interface = gr.ChatInterface(
    fn=generate_recipe_and_image, 
    type="messages",
    title="AI Recipe Generator",
    description="Enter your ingredients, and the AI will generate a detailed recipe along with an image of the dish.",
    theme=gr.themes.Citrus(),
    examples=["chicken, rice, tomatoes, onions, spices", "oats, milk, fruits", "Noodles, Tofu, Soy sauce"],
)

In [10]:
chat_interface.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


type(recipe_text): <class 'str'>
recipe_text: Recipe: Spicy Chicken and Tomato Rice

Ingredients:
- 4 boneless, skinless chicken thighs (or breasts)
- 2 cups long-grain rice
- 1 can (15 ounces) diced tomatoes, drained (or 2 fresh tomatoes, chopped)
- 1 large onion, finely chopped
- 2 tablespoons olive oil
- 3 cups chicken broth
- 1 teaspoon ground cumin
- 1 teaspoon smoked paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and black pepper, to taste
- Fresh cilantro or parsley, chopped (for garnish, optional)
- Lemon wedges (for serving, optional)

Cooking Time: 50 minutes

Instructions:

1. **Prepare the Chicken:**
   - Pat the chicken thighs (or breasts) dry with paper towels. Season both sides generously with salt, black pepper, and 1/2 teaspoon of cumin, paprika, and cayenne pepper.

2. **Brown the Chicken:**
   - In a large, deep skillet or Dutch oven, heat 1 tablespoon of olive oil over medium-high heat.
   - Once the oil is hot, add the chicke

/opt/anaconda3/envs/SDS_Projects/lib/python3.12/site-packages/gradio/blocks.py:1780: UserWarning: A function (_submit_fn) returned too many output values (needed: 2, returned: 3). Ignoring extra values.
    Output components:
        [state, chatbot]
    Output values returned:
        ["Recipe: Spicy Chicken and Tomato Rice

Ingredients:
- 4 boneless, skinless chicken thighs (or breasts)
- 2 cups long-grain rice
- 1 can (15 ounces) diced tomatoes, drained (or 2 fresh tomatoes, chopped)
- 1 large onion, finely chopped
- 2 tablespoons olive oil
- 3 cups chicken broth
- 1 teaspoon ground cumin
- 1 teaspoon smoked paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and black pepper, to taste
- Fresh cilantro or parsley, chopped (for garnish, optional)
- Lemon wedges (for serving, optional)

Cooking Time: 50 minutes

Instructions:

1. **Prepare the Chicken:**
   - Pat the chicken thighs (or breasts) dry with paper towels. Season both sides generously with